In [ ]:
import os
from config.config import config
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

def get_model():
    os.environ["OPENAI_API_KEY"] = config.get("apikey", "openai")
    model = ChatOpenAI(model="gpt-4o-mini")
    return model

store = {} # 대화 내역을 저장할 store

def get_session_history(session_id:str) -> BaseChatMessageHistory: # session_id : 대화 이력을 반환받고자 하는 세션의 식별자
    # 만약 session store에 없는 session_id 라면
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory() # 메모리에 있는 대화 기록을 저장하는 객체 생성
    return store[session_id] # 해당 세션의 대화 기록을 반환하도록 함

In [ ]:
model = get_model()
with_message_history = RunnableWithMessageHistory(model, get_session_history) # --- 대화 내용을 저장하면서 대화를 이어나가는 클래스

In [ ]:
config = {"configurable": {"session_id":"abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 나는 Jongya야.")],
    config = config
)

print(response.content)

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐라고?")],
    config = config
)

print(response.content)

In [ ]:
# 세션 아이디를 바꾸면 이전 대화내용을 모름  
config = {
    "configurable" : {"session_id":"ccc1"}
}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐라고?")],
    config = config
)

print(response.content)

In [ ]:
# store를 출력해보면  
from pprint import pprint

pprint(store)

In [ ]:
config = {"configurable": {"session_id":"abc2"}}

for r in with_message_history.stream(
    [HumanMessage(content="내가 어느 나라 사람인지 맞춰보고, 그 나라의 국가를 불러줘.")],
    config = config):
    print(r.content, end="|")

